In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# pip install timm

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import sklearn
import kagglehub
import random
from tqdm import tqdm
import timm

In [4]:
print(f"OS version: {os.name}")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")
import matplotlib
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Scikit-Learn version: {sklearn.__version__}")
print(f"Kagglehub version: {kagglehub.__version__}")

OS version: posix
OpenCV version: 4.11.0
NumPy version: 1.26.4
Pandas version: 2.2.3
PyTorch version: 2.6.0+cu124
TorchVision version: 0.21.0+cu124
Matplotlib version: 3.7.2
Scikit-Learn version: 1.2.2
Kagglehub version: 0.3.12


In [5]:
def label_from_path(path):
    return "real" if "original" in path else "fake"

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
print(DEVICE)

cuda


In [8]:
!nvidia-smi

Sat Jul 12 05:24:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             27W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
torch.manual_seed(42)
random.seed(42)


In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("noobcoder27/processed-faces")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/processed-faces


In [11]:
base = '/kaggle/input/processed-faces/processed_faces'

In [12]:
def load_images_and_labels(split): 
    image_paths, labels = [], [] # image_path -> stores the image path of split dataset and labels[] -> corresponding labels 
    for label in ['real', 'fake']:
        directory = f'{base}/{split}/{label}'
        if not os.path.exists(directory): continue
        for root, _, files in os.walk(directory):
            for fname in files:
                if fname.endswith('.jpg'):
                    image_paths.append(os.path.join(root, fname))
                    labels.append(0 if label == 'real' else 1)
    return image_paths, labels

In [13]:
X_train, y_train = load_images_and_labels('train') # 'processed_faces/train/real/video1/0.jpg'

In [14]:
len(X_train),len(y_train)

(6710, 6710)

In [15]:
X_val, y_val = load_images_and_labels('val')

In [16]:
print(len(X_val),len(y_val))

1500 1500


In [17]:
X_test, y_test = load_images_and_labels('Test')

In [18]:
print(len(X_test),len(y_test))

1480 1480


In [19]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import glob
import random

class customDataset(Dataset):
    def __init__(self, base_dir, split='train', sequence_len=10):
        self.sequence_len = sequence_len
        self.samples = []
        self.split = split

        for label in ['real', 'fake']:
            folder = os.path.join(base_dir, split, label)
            if not os.path.exists(folder):
                continue

            for video_folder in os.listdir(folder):
                path = os.path.join(folder, video_folder)
                if os.path.isdir(path):
                    frames = sorted(glob.glob(os.path.join(path, '*.jpg')))
                    if len(frames) >= sequence_len:
                        self.samples.append((frames, 0 if label == 'real' else 1))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        frame_paths, label = self.samples[idx]
        chosen = sorted(random.sample(frame_paths, self.sequence_len))
        frames = []

        for fp in chosen:
            img = Image.open(fp).convert("RGB")

            if label == 0 and self.split == 'train':
                img = train_real_transforms(img)
            elif label == 1 and self.split == 'train':
                img = train_fake_transforms(img)
            else:
                img = val_test_transforms(img)

            frames.append(img)

        frames = torch.stack(frames)  # Shape: [T, 3, 224, 224]
        return frames, torch.tensor(label, dtype=torch.long)


In [20]:
from torchvision import transforms

# For real images in training
train_real_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])


train_fake_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

In [21]:
BATCH_SIZE = 8 # No. of images in a batch ie 128 images in a single batch
EPOCHS = 10 # Try increasing epochs to 30
LEARNING_RATE = 3e-4
PATCH_SIZE = 32 # (P,P)
NUM_CLASSES = 2
IMAGE_SIZE = 224 # Transform the image and make the size go to 224
CHANNELS = 3
EMBED_DIM = 256
NUM_HEADS = 4 # INcrease the number heads
DEPTH = 6 # No. of encoder layers
MLP_DIM = 512
DROP_RATE = 0.1

In [22]:
train_dataset = customDataset(base, split='train', sequence_len=10)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [23]:
print(f"Loaded {len(train_dataset)} samples in train dataset")

Loaded 671 samples in train dataset


In [24]:
len(train_dataset) # to detect how many videos are having 10 frames which are used for training 

671

In [25]:
val_dataset   = customDataset(base, split='val', sequence_len=10)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [26]:
print(len(val_dataset))

150


In [27]:
test_dataset  = customDataset(base, split='Test', sequence_len=10)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [28]:
print(len(test_dataset))

148


In [29]:
base

'/kaggle/input/processed-faces/processed_faces'

In [30]:
class TemporalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads=4, num_layers=2):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.cls_token = nn.Parameter(torch.randn(1, 1, input_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, 500, input_dim))

    def forward(self, x):  # x: [B, T, D]
        B, T, D = x.shape
        cls_tokens = self.cls_token.expand(B, 1, D)
        x = torch.cat([cls_tokens, x], dim=1)  # [B, T+1, D]
        x = x + self.pos_embedding[:, :T+1, :]
        x = x.permute(1, 0, 2)  # [T+1, B, D]
        x = self.transformer(x)
        return x[0]  # CLS token output


In [31]:
len(train_dataset)

671

In [32]:
# Let's check out what we've created
print(f"DataLoader: {train_loader,val_loader, test_loader}")
print(f"Length of train_loader: {len(train_loader)} batches of {BATCH_SIZE}...")
print(f"Length of train_loader: {len(val_loader)} batches of {BATCH_SIZE}...")
print(f"Length of test_loader: {len(test_loader)} batches of {BATCH_SIZE}...")

DataLoader: (<torch.utils.data.dataloader.DataLoader object at 0x7a4ed8094510>, <torch.utils.data.dataloader.DataLoader object at 0x7a4ed6358150>, <torch.utils.data.dataloader.DataLoader object at 0x7a4ed61dbf90>)
Length of train_loader: 84 batches of 8...
Length of train_loader: 19 batches of 8...
Length of test_loader: 19 batches of 8...


In [33]:
21*32

672

In [34]:
5*32

160

In [35]:
class PatchEmbedding(nn.Module):
    def __init__(self,
                 img_size,
                 patch_size,
                 in_channels,
                 embed_dim):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_channels=in_channels,
                              out_channels=embed_dim,
                              kernel_size=patch_size,
                              stride=patch_size)
        num_patches = (img_size // patch_size) ** 2
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim)) # [Batch singleton, 1 CLS token for every image, Embedding Dimension]
        self.pos_embed = nn.Parameter(torch.randn(1, 1 + num_patches, embed_dim)) # [Batch singleton, Token positions (1 for [CLS] token + num_patches for image patches), Embedding dimension]

    def forward(self, x: torch.Tensor):
        B = x.size(0) # x -> (B, C, H, W)
        x = self.proj(x) # (B, E, H/P, W/P) ie for every image in B( batch size) divided into no. of patches (H/P,W/P) and each patch has output dime after projection as Embedding Dimension
        x = x.flatten(2).transpose(1, 2) # flatten(H/P, W/P) -> N (No of patches) -> (B, E, N) -> transpose -> (B, N, E)
        cls_tokens = self.cls_token.expand(B, -1, -1) # (1, 1, E) -> (B, 1, E) Expands the learned [CLS] token to batch size B
        # x: (B,C,H,W) -> (B,E,H/P,W/P) -> (B,N,E), cls_token: (1,1,E)->(B,1,E)
        x = torch.cat((cls_tokens, x), dim=1) # x: (B,N,E) & cls_token: (B,1,E) -> (B,1+N,E)embed a [CLS] token with every image(containing N patches)
        x = x + self.pos_embed # (B,1+N,E) -> (B,1+N,E) actually pos_embed is broadcasted to B batches from 1 batch
        return x # (B,1+N,E)

In [36]:
class MLP(nn.Module):
    def __init__(self,
                 in_features,
                 hidden_features,
                 drop_rate):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.GELU(),
            nn.Dropout(p=drop_rate),
            nn.Linear(hidden_features, in_features),
            nn.Dropout(p=drop_rate)
        )

    def forward(self, x):
        x = self.network(x)
        return x

In [37]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, drop_rate):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=drop_rate, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = MLP(embed_dim, mlp_dim, drop_rate)

    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

In [38]:
# class DeepFakeDetector(nn.Module):
#     def __init__(self,
#                  img_size,
#                  patch_size,
#                  in_channels,
#                  num_classes,
#                  embed_dim,
#                  depth,
#                  num_heads,
#                  mlp_dim,
#                  drop_rate,
#                  temporal_layers=2,
#                  temporal_heads=4):
#         super().__init__()

#         # Vision Transformer backbone (no final head)
#         self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
#         self.encoder = nn.Sequential(*[
#             TransformerEncoderLayer(embed_dim, num_heads, mlp_dim, drop_rate)
#             for _ in range(depth)
#         ])
#         self.norm = nn.LayerNorm(embed_dim)

#         # Temporal transformer for sequence modeling
#         self.temporal_transformer = TemporalTransformer(
#             input_dim=embed_dim,
#             num_heads=temporal_heads,
#             num_layers=temporal_layers
#         )

#         # Final classification head (after temporal aggregation)
#         self.head = nn.Linear(embed_dim, num_classes)

#     def forward(self, x):
#         """
#         x: Tensor of shape [B, T, C, H, W]
#         """
#         B, T, C, H, W = x.shape
    
#         # Reshape to feed each frame through ViT individually
#         x = x.view(B * T, C, H, W)  # [B*T, C, H, W]
    
#         # Pass through patch embedding and transformer encoder
#         x = self.patch_embed(x)     # [B*T, 1+N, D]
#         x = self.encoder(x)         # [B*T, 1+N, D]
#         x = self.norm(x)            # [B*T, 1+N, D]
#         cls_tokens = x[:, 0]        # Extract CLS token: [B*T, D]
    
#         # Reshape back to video format
#         cls_tokens = cls_tokens.view(B, T, -1)  # [B, T, D]
    
#         # Temporal modeling
#         video_repr = self.temporal_transformer(cls_tokens)  # [B, D]
    
#         # Classification
#         out = self.head(video_repr)  # [B, num_classes]
#         return out


In [39]:
import torch
import torch.nn as nn
import timm

class DeepFakeDetector(nn.Module):
    def __init__(self, pretrained_model='vit_base_patch16_224', num_classes=2, temporal_layers=2, temporal_heads=4):
        super().__init__()
        
        # Load pretrained ViT from timm (no head)
        self.vit = timm.create_model(pretrained_model, pretrained=True)
        self.vit.head = nn.Identity()  # Remove classification head

        # Freeze ViT 
        for param in self.vit.parameters():
            param.requires_grad = False

        self.temporal = TemporalTransformer(
            input_dim=self.vit.embed_dim,
            num_heads=temporal_heads,
            num_layers=temporal_layers
        )

        self.head = nn.Linear(self.vit.embed_dim, num_classes)

    def forward(self, x):
        """
        x shape: [B, T, C, H, W] where T = number of frames
        """
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)               # [B*T, C, H, W]
        features = self.vit(x)                   # [B*T, D]
        features = features.view(B, T, -1)       # [B, T, D]

        temporal_out = self.temporal(features)   # [B, D]
        return self.head(temporal_out)           # [B, num_classes]


In [40]:
model = DeepFakeDetector().to(DEVICE)
# model = nn.DataParallel(model)
# model = model.to(DEVICE)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [41]:
optimizer = torch.optim.Adam([
    {'params': model.parameters(), 'lr': LEARNING_RATE}
])

In [42]:
loss_fn = nn.CrossEntropyLoss()
epochs = EPOCHS

In [43]:
def train(model, loader, optimizer, criterion):
    # Set the mode of the model into training
    model.train()
    device = DEVICE
    total_loss, correct = 0, 0

    for x, y in loader:
        # Moving (Sending) our data into the target device
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        # 1. Forward pass (model outputs raw logits)
        out = model(x)
        # 2. Calcualte loss (per batch)
        loss = criterion(out, y)
        # 3. Perform backpropgation
        loss.backward()
        # 4. Perforam Gradient Descent
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        correct += (out.argmax(1) == y).sum().item()
    # You have to scale the loss (Normlization step to make the loss general across all batches)
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

In [44]:
def evaluate(model, loader):
    model.eval() # Set the mode of the model into evlauation
    correct = 0
    device = DEVICE
    with torch.inference_mode():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            correct += (out.argmax(dim=1) == y).sum().item()
    return correct / len(loader.dataset)

In [45]:
from tqdm.auto import tqdm

In [46]:
### Training
train_accuracies, test_accuracies = [], []

for epoch in tqdm(range(EPOCHS)):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    print(f"Epoch: {epoch+1}/{EPOCHS}, Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}%, Test acc: {test_acc:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1/10, Train loss: 0.8850, Train acc: 0.5127%, Test acc: 0.4932
Epoch: 2/10, Train loss: 0.7359, Train acc: 0.5663%, Test acc: 0.5203
Epoch: 3/10, Train loss: 0.6193, Train acc: 0.6453%, Test acc: 0.5338
Epoch: 4/10, Train loss: 0.5767, Train acc: 0.7034%, Test acc: 0.5405
Epoch: 5/10, Train loss: 0.6959, Train acc: 0.6080%, Test acc: 0.5068
Epoch: 6/10, Train loss: 0.7154, Train acc: 0.4858%, Test acc: 0.5068
Epoch: 7/10, Train loss: 0.6793, Train acc: 0.5350%, Test acc: 0.5000
Epoch: 8/10, Train loss: 0.7345, Train acc: 0.5410%, Test acc: 0.4932
Epoch: 9/10, Train loss: 0.6608, Train acc: 0.5872%, Test acc: 0.4932
Epoch: 10/10, Train loss: 0.7083, Train acc: 0.5306%, Test acc: 0.4932


### Ignore

In [47]:
def train_one_epoch(model,train_loader,optimizer,loss_fn):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch_features, batch_labels in train_loader:
            batch_features = batch_features.to(DEVICE)
            batch_labels = batch_labels.to(DEVICE)
            optimizer.zero_grad()
            y_pred = model(batch_features)               # [B, 2]
            loss = loss_fn(y_pred, batch_labels)         # labels: [B]
            _, predicted = torch.max(y_pred, 1) 
            loss.backward()
            optimizer.step()
            total_loss += loss.item()         # [B]
            correct += (predicted == batch_labels).sum().item()
            total += batch_labels.size(0)
    
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total
    return avg_loss,accuracy

In [48]:
def evaluate_loss_and_accuracy(model, loader, loss_fn):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            _,predicted = torch.max(y_pred,dim=1)
            total_loss += loss.item() 
            correct += (predicted == y).sum().item()
            total += y.size(0)

    avg_loss = total_loss / len(loader)
    accuracy = correct / total
    return avg_loss, accuracy

In [49]:
from sklearn.metrics import balanced_accuracy_score
import torch
import os

def train_model(model, train_loader, val_loader, test_loader=None, optimizer=None, loss_fn=None,
                epochs=20, model_save_path='best_model.pth'):
    
    train_losses, val_losses, val_accuracies = [], [], []
    best_val_acc = 0
    best_model_state = None

    for epoch in range(epochs):
        # Train for one epoch
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, loss_fn)
        
        # Validation loss and accuracy
        val_loss, val_acc = evaluate_loss_and_accuracy(model, val_loader, loss_fn)

        # Logging
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}: "
              f"Train Loss = {train_loss:.4f}, "
              f"Val Loss = {val_loss:.4f}, "
              f"Val Acc = {val_acc:.4f}")

        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()
            torch.save(best_model_state, model_save_path)
            print(f" New best model saved with Val Acc = {val_acc:.4f}")

        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

    # Load best saved model after all epochs
    if os.path.exists(model_save_path):
        model.load_state_dict(torch.load(model_save_path))

    # Plot learning curves
    plot_learning_curves(train_losses, val_losses, val_accuracies)

    # # Tune threshold using validation set
    # val_probs, val_targets = get_predictions_and_targets(model, val_loader)
    # best_thresh = find_best_threshold(val_targets, val_probs)
    # return best_thresh


In [50]:
import matplotlib.pyplot as plt

def plot_learning_curves(train_losses, val_losses, val_accuracies):
    epochs_range = range(1, len(train_losses) + 1)
    plt.figure(figsize=(14, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, label='Train Loss')
    plt.plot(epochs_range, val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.grid(True)
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, val_accuracies, label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy Curve')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=20
)

Epoch 1: Train Loss = 0.6662, Val Loss = 0.7135, Val Acc = 0.5600
 New best model saved with Val Acc = 0.5600
Epoch 2: Train Loss = 0.6891, Val Loss = 0.6215, Val Acc = 0.6000
 New best model saved with Val Acc = 0.6000
Epoch 3: Train Loss = 0.6328, Val Loss = 0.6353, Val Acc = 0.6267
 New best model saved with Val Acc = 0.6267
Epoch 4: Train Loss = 0.5802, Val Loss = 0.7164, Val Acc = 0.5600
Epoch 5: Train Loss = 0.5626, Val Loss = 1.3041, Val Acc = 0.5067
Epoch 6: Train Loss = 0.5481, Val Loss = 1.1752, Val Acc = 0.5200
Epoch 7: Train Loss = 0.5908, Val Loss = 0.9071, Val Acc = 0.5133
Epoch 8: Train Loss = 0.5457, Val Loss = 1.3478, Val Acc = 0.5200
Epoch 9: Train Loss = 0.5513, Val Loss = 0.8856, Val Acc = 0.5067
Epoch 10: Train Loss = 0.5215, Val Loss = 1.2916, Val Acc = 0.5133
Epoch 11: Train Loss = 0.4997, Val Loss = 1.4342, Val Acc = 0.5400
Epoch 12: Train Loss = 0.5815, Val Loss = 0.8749, Val Acc = 0.5200
Epoch 13: Train Loss = 0.6139, Val Loss = 0.6886, Val Acc = 0.5333
Epoch 

### Test Dataset

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    device = DEVICE
    y_true, y_pred, y_probs = [], [], []
    for batch_features,batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        prob = model(batch_features)
        _,pred = torch.max(prob,dim=1)
        correct += (pred == batch_labels).sum().item()
        total += batch_labels.size(0)
        y_true.append(batch_labels.cpu().numpy())
        y_probs.append(prob.cpu().numpy())
        y_pred.append(pred.cpu().numpy())
        # print(total)
    print(f'Test Accuracy: {correct/total:.4f}')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure proper format
y_true = np.array(y_true).astype(int).ravel()
y_pred = np.array(y_pred).astype(int).ravel()

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Real", "Fake"],
            yticklabels=["Real", "Fake"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix on Test Set")
plt.show()

In [ ]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    balanced_accuracy_score,
    classification_report,
    confusion_matrix,
    matthews_corrcoef
)
balanced_acc = balanced_accuracy_score(y_true, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.4f}")

In [ ]:
cm = confusion_matrix(y_true, y_pred)
TN, FP, FN, TP = cm.ravel()

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

gmean = np.sqrt(sensitivity * specificity)
print(f"G-mean: {gmean:.4f}")

In [ ]:
mcc = matthews_corrcoef(y_true, y_pred)
print(f"MCC: {mcc:.4f}")